## **Section 0 - Important Information**

1. Documentation
    1. Phidget22 Python Libraries
        * Link: https://www.phidgets.com/?view=api
        * *Set language to Python*
        * *Set API to one of the 3 this file uses: Phidget, DigitalInput, and Stepper*
    2. X-Winder User Manual
        * Link: https://docs.google.com/document/d/1Ye56XaXfxHr0qRkJA9x0qhNSYwfYPzcqx89bb03exbQ/edit?usp=sharing
        * *Must have access to RP's Google Drive to view this document. Reach out to the RP President, Vice President, or your section manager if you don't have access*
        * *Currently unfinished*
2. Notes 
    * All of the code in this file is written in Python (runs on version 3.9.11)
    * This document contains 3 sections:
        1. Basic Initializations
        2. Function Initializations
        3. X-Winder Control Panel
    * For sections 1 & 2, make sure to run all of the cells in order for proper code setup

## **Section 1 - Basic Initializations**

Step 1.1 - Import Libraries

In [ ]:
from Phidget22.Phidget import *
from Phidget22.Devices.DigitalInput import *
from Phidget22.Devices.Stepper import *
from numpy import pi, degrees, radians, tan, cos, arccos
from time import sleep

Step 1.2 - Set One-Time User Parameters (User Modifiable)

In [ ]:
##################################################
# MOTOR PARAMETERS
##################################################

mandrel_serial_num  = 615813 
carriage_serial_num = 615519 
head_serial_num     = 616004
motor_timeout       = 5000   # (ms)

##################################################
# HELICAL-WINDING + HOOP-WINDING PARAMETERS
##################################################

travel_distance = 22.5 # [desired tube length + 2*diameter of mandrel]? (in) 

##################################################
# HELICAL-WINDING PARAMETERS
##################################################

helical_fiber_bandwidth = 0.16 # (in)
user_alpha              = 55   # suggested winding angle (deg) 
radius                  = 0.25 # mandrel outer radius (in)
L                       = 1.6  # linear speed of the carriage (in/s) 
offset_fudge_factor     = 1    # offset angle multiplier (%)
head_to_90_distance     = 0.25 # distance needed for head to rotate from alpha deg to 90 deg for bird's nest (in)

##################################################
# HOOP-WINDING PARAMETERS
##################################################

hoop_fiber_bandwidth = 0.152 # (in)
linear_velocity      = 0.2   # velocity of carriage (in/s)

Step 1.3 - Connect to X-Winder Motors

In [ ]:
# motor for rotating mandrel   
mandrel = Stepper()
mandrel.setDeviceSerialNumber(mandrel_serial_num) 
mandrel.openWaitForAttachment(motor_timeout)
mandrel.setAcceleration(40000)
mandrel.setRescaleFactor(0.01688) # figure rescale factors out
mandrel.setCurrentLimit(4)

# motor for moving the carriage back and forth alongside the mandrel
carriage = Stepper()
carriage.setDeviceSerialNumber(carriage_serial_num) 
carriage.openWaitForAttachment(motor_timeout)
carriage.setAcceleration(20000)
carriage.setRescaleFactor(-0.00070)
carriage.setCurrentLimit(4)

# motor for rotating the head of the carriage to change the fiber angle
head = Stepper()
head.setDeviceSerialNumber(head_serial_num) 
head.openWaitForAttachment(motor_timeout)
head.setAcceleration(4800)
head.setRescaleFactor(0.02641)
head.setCurrentLimit(3)

Step 1.4 - Run Calculations for Helical-Winding and Hoop-Winding

In [ ]:
##################################################
# FUNCTION DEFINITIONS
##################################################

# helical-winding N to alpha calculation function
# based on helix/triangle calculations
# inputs:  
#   N = number of winding iterations
#   B = helical_fiber_thickness (in)
#   D = mandrel outer diameter (in)
# outputs:
#   calc_deg = calculated alpha based on N (deg)
def calc_alpha(N, B, D):
    alpha_rad = arccos( (N*B)/(pi*D) ) # (rad)
    alpha_deg = degrees(alpha_rad)      # (rad --> deg)
    return alpha_deg               

##################################################
# HELICAL-WINDING CALCULATIONS
################################################## 

N = 5        # number of iterations needed to cover mandrel, initial guess?
D = 2*radius # mandrel outer diameter (in)
B = helical_fiber_bandwidth

# while loop parameters
# values don't matter, just that prev_alpha_error > alpha_error initially
alpha_error      = 0 #    
prev_alpha_error = 1 #

# find actual value of N and alpha using user_alpha
while (alpha_error < prev_alpha_error):
    N += 1
    alpha_error = abs( calc_alpha(N, B, D) - user_alpha )
    prev_alpha_error = abs( calc_alpha(N-1, B, D) - user_alpha )

# final alpha value
alpha_deg = calc_alpha(N-1, B, D) # (deg)
alpha_rad = radians(alpha_deg)    # (deg --> rad)

angular_offset = ( (B*360)/(pi*D*cos(alpha_rad)) )*offset_fudge_factor # 360 deg + small value rotation of mandrel (deg/s)

mandrel_W = (L*tan(alpha_rad)*360)/(pi*D) # angular velocity of mandrel (deg/s)
head_W = (90-alpha_deg)/(360/mandrel_W)*1.5 # angular velocity of head (deg/s)

forward_time = travel_distance/L # time for 1 forward iteration? (s)
total_angle_change = mandrel_W*forward_time # tetha change between 1 forward pass 
head_F = -(90-(alpha_deg+15)) # when head is perpendicular to fiber during forwards winding (deg)
head_B = 90-(alpha_deg+15) # when head is perpendicular to fiber during backwards winding (deg)

print('Verify Helical-Winding Calculations')
print(f'1.) number of iterations: {N}')
print(f'2.) final alpha: {alpha_deg} deg')
print(f'3.) angular offset: {angular_offset} deg')

##################################################
# HOOP-WINDING CALCULATIONS
##################################################

time = hoop_fiber_bandwidth/linear_velocity # (s)
angular_velocity = 360/time                 # (deg/s?)

## **Section 2 - Function Initializations**

Step 2.1 - Initialize Helical-Winding Sub Functions

In [ ]:
def mandrel_moving():
    while (mandrel.getIsMoving()): 
        sleep(0.5)

def helical_path_forwards():
    head.setTargetPosition(head_F)
    head.setEngaged(True)

    carriage.setTargetPosition(travel_distance)
    mandrel.setTargetPosition( mandrel.getTargetPosition()-total_angle_change )

    head.setEngaged(True)
    mandrel.setEngaged(True)
    carriage.setEngaged(True)

def taper():
    head.setVelocityLimit(head_W)
    head.setTargetPosition(0)
    mandrel.setTargetPosition( mandrel.getTargetPosition()-360 )

    mandrel.setEngaged(True)
    head.setEngaged(True)

def move_head_to_helical_angle_b():
    head.setVelocityLimit( abs(head_B)/(head_to_90_distance/L) )
    head.setTargetPosition(head_B)
       
    carriage.setTargetPosition(0)
    mandrel.setTargetPosition( mandrel.getTargetPosition()-total_angle_change )

    head.setEngaged(True)
    mandrel.setEngaged(True)
    carriage.setEngaged(True)

def turn_head_while_tapering():
    head.setVelocityLimit(head_W)
    head.setTargetPosition(0)
    mandrel.setTargetPosition( mandrel.getTargetPosition()-360-angular_offset )

    mandrel.setEngaged(True)
    head.setEngaged(True)

def move_head_to_helical_angle_f():
    head.setVelocityLimit( abs(head_B)/(head_to_90_distance/L) )
    head.setTargetPosition(head_F)

# define forward and backward sub-function run order(s) 
forward_sequence  = [
                    helical_path_forwards,
                    mandrel_moving,
                    taper,
                    mandrel_moving,
                    move_head_to_helical_angle_b,
                    mandrel_moving,
                    turn_head_while_tapering,
                    mandrel_moving,
                    move_head_to_helical_angle_f
                    ]
backward_sequence = [
                    move_head_to_helical_angle_b,
                    mandrel_moving,
                    turn_head_while_tapering,
                    mandrel_moving,
                    move_head_to_helical_angle_f,
                    helical_path_forwards,
                    mandrel_moving,
                    taper,
                    mandrel_moving
                    ]

Step 2.2 - Initialize Helical-Winding Main Function 

In [ ]:
#######################################################################################
# 1. NAME: 
#    - start_helical_winding
# 2. DESCRIPTION:
#    - content here
# 3. INPUTS
#    - forwards (boolean)
#      * True  = forwards helical-winding direction  (position: 0 --> travel_distance)
#      * False = backwards helical-winding direction (position: travel_distance --> 0)
# 4. OUTPUTS: 
#    - integer
#      * error   = -1
#      * success =  0
#######################################################################################

def start_helical_winding(forwards):
    # set variables based on hoop-winding direction
    # if neither True or False, exit the function
    if (forwards == True):
        run_sequence = forward_sequence
    elif (forwards == False):
        run_sequence = backward_sequence
    else:
        print('INVALID USER PARAMETERS')
        return -1

    # notify user helical-winding has started
    if (forwards == True):
        print('FORWARDS HELICAL-WINDING IN PROGRESS...')
    elif (forwards == False):
        print('BACKWARDS HELICAL-WINDING IN PROGRESS...')

    ##################################################
    # START OF HELICAL-WINIDING LOGIC
    ##################################################

    carriage.setVelocityLimit(L)
    mandrel.setVelocityLimit(mandrel_W)

    # iterate through run_sequence N times to cover the entire mandrel
    for i in range(N):
        # for each iteration, run each function in run_sequence in order
        for j in range(len(run_sequence)):
            run_sequence[i]()
    
    ##################################################
    # END OF HELICAL-WINIDING LOGIC
    ##################################################

    # notify user hoop-winding has finished
    # fix timing
    if (forwards == True):
        print('FORWARDS HELICAL-WINDING FINISHED')
    elif (forwards == False):
        print('BACKWARDS HELICAL-WINDING FINISHED')

    # exit function (success)
    return 0

Step 2.3 - Initialize Hoop-Winding Main Function 

In [ ]:
####################################################################################
# 1. NAME: 
#    - start_hoop_winding
# 2. DESCRIPTION:
#    - content here
# 3. INPUTS
#    - forwards (boolean)
#      * True  = forwards hoop-winding direction  (position: 0 --> travel_distance)
#      * False = backwards hoop-winding direction (position: travel_distance --> 0)
# 4. OUTPUTS: 
#    - integer
#      * error   = -1
#      * success =  0
####################################################################################

def start_hoop_winding(forwards):
    # set variables based on hoop-winding direction
    # if neither True or False, exit the function
    if (forwards == True):
        carriage_target_position = travel_distance
    elif (forwards == False):
        carriage_target_position = 0
    else:
        print('INVALID USER PARAMETERS')
        return -1

    # notify user hoop-winding has started
    if (forwards == True):
        print('FORWARDS HOOP-WINDING IN PROGRESS...')
    elif (forwards == False):
        print('BACKWARDS HOOP-WINDING IN PRORESS...')

    ##################################################
    # START OF HOOP-WINIDING LOGIC
    ##################################################

    carriage.setVelocityLimit(linear_velocity)
    mandrel.setVelocityLimit(angular_velocity)

    # target position == end position 
    head.setTargetPosition(0)
    carriage.setTargetPosition(carriage_target_position)
    mandrel.setTargetPosition( mandrel.getTargetPosition()-(travel_distance*(360/hoop_fiber_bandwidth)) )

    # start motor movement
    head.setEngaged(True)
    mandrel.setEngaged(True)
    carriage.setEngaged(True)
    
    ##################################################
    # END OF HOOP-WINIDING LOGIC
    ##################################################

    # notify user hoop-winding has finished
    # fix timing
    if (forwards == True):
        print('FORWARDS HOOP-WINDING FINISHED')
    elif (forwards == False):
        print('BACKWARDS HOOP-WINDING FINISHED')

    # exit function (success)
    return 0

## **Section 3 - X-Winder Control Panel**

1.) Run Helical-Winding Forwards 

In [ ]:
start_helical_winding(forwards=True)

2.) Run Helical-Winding Backwards

In [ ]:
start_helical_winding(forwards=False)

3.) Run Hoop-Winding Forwards

In [ ]:
start_hoop_winding(forwards=True)

4.) Run Hoop-Winding Backwards

In [ ]:
start_hoop_winding(forwards=False)

5.) Disconnect Motors (For Manual Adjustment)

In [ ]:
mandrel.setEngaged(0)
carriage.setEngaged(0)
head.setEngaged(0)

# potentially needed?
# head.close()
# mandrel.close()
# carriage.close()